In [1]:
import torch
from data_handlers import Load_ImageNet100
from overcomplete.models import DinoV2, ViT, ResNet
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import TopKSAE, train_sae
from overcomplete.visualization import (overlay_top_heatmaps, evidence_top_images, zoom_top_images, contour_top_image)
import os
import matplotlib.pyplot as plt
from einops import rearrange
from universal_trainer import train_usae
from activation_generator import Load_activation_dataloader
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
models = {
    "DinoV2": DinoV2(device="cuda"),
    "ViT": ViT(device="cuda")
}

Using cache found in C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def compute_normalization_stats(activations):
    # Flatten across batch and sequence
    flat = activations.permute(1, 0, 2).reshape(activations.shape[1], -1)  # (D, N*T)
    mean = flat.mean(dim=1)  # (D,)
    std = flat.std(dim=1)  # (D,)
    print("Mean: ", mean)
    print("Std: ", std)
    return mean, std

In [ ]:
image_loader = Load_ImageNet100(transform=None, batch_size=1024, shuffle=True)

activations = models["DinoV2"].forward_features(img.to(device))
print("Activations Shape: ", activations.shape)


Activations Shape:  torch.Size([1024, 256, 384])


NameError: name 'compute_normalization_stats' is not defined

In [ ]:
# Compute normalization stats
mean, std = compute_normalization_stats(activations)

Mean:  tensor([-0.0111, -0.0060, -0.0070, -0.0059, -0.0114, -0.0088, -0.0110, -0.0056,
        -0.0050, -0.0037,  0.0015,  0.0026,  0.0033,  0.0024,  0.0014, -0.0064,
         0.0036,  0.0058,  0.0103,  0.0048,  0.0056,  0.0044,  0.0075,  0.0062,
         0.0086,  0.0153,  0.0136,  0.0219,  0.0153,  0.0188,  0.0129,  0.0098,
         0.0129,  0.0151,  0.0168,  0.0147,  0.0165,  0.0122,  0.0137,  0.0156,
         0.0209,  0.0279,  0.0254,  0.0340,  0.0244,  0.0221,  0.0205,  0.0172,
         0.0172,  0.0190,  0.0252,  0.0196,  0.0212,  0.0203,  0.0222,  0.0228,
         0.0266,  0.0342,  0.0318,  0.0361,  0.0287,  0.0343,  0.0248,  0.0217,
         0.0218,  0.0249,  0.0329,  0.0281,  0.0306,  0.0266,  0.0275,  0.0276,
         0.0342,  0.0403,  0.0397,  0.0446,  0.0370,  0.0379,  0.0328,  0.0283,
         0.0234,  0.0266,  0.0345,  0.0317,  0.0335,  0.0307,  0.0313,  0.0318,
         0.0367,  0.0422,  0.0438,  0.0470,  0.0402,  0.0438,  0.0363,  0.0302,
         0.0276,  0.0313,  0.0371